# **8 Model Performance**

In [31]:
# o erro da função é que a pessoa precisa passar um dataset com a variável target chamada de Response

def precision_at_k(data,k=2000):   #2000 é apenas o default caso ngm passe o valor, poderia ser outro valor como 20000
    # reset index
    data=data.reset_index(drop=True) #necessário para q o index nao seja igual aos ids

    # create ranking order
    data['ranking']=data.index+1    #(para ñ começar do zero)

    data['precision_at_k']=data['response'].cumsum()/data['ranking']
    
    return data.loc[k,'precision_at_k']

In [36]:
def recall_at_k(data,k=2000):   #k poderia ser outro valor como 20000
    # reset index
    data=data.reset_index(drop=True) #necessário para q o index nao seja igual aos ids

    # create ranking order
    data['ranking']=data.index+1    #(para ñ começar do zero)

    data['recall_at_k']=data['response'].cumsum() /data['response'].sum()
    
    return data.loc[k,'recall_at_k']

In [26]:
# copy_data
df8=x_validation.copy()
df8['response']=y_validation.copy()       #response = propensity score

# propensity score
df8['score']=yhat_knn[:,1].tolist()

# sorte clients by propensity score
df8=df8.sort_values('score',ascending=False)

# compute precision at k

# compute recall at k

In [27]:
df8.head()

,id,previously_insured,annual_premium,vintage,gender,age,region_code,policy_sales_channel,driving_license,vehicle_damage,vehicle_age_below_1_year,vehicle_age_between_1_2_years,vehicle_age_over_2_years,response,score
110249,110250,0,1.058370,0.259516,0.138422,0.415385,0.187506,0.209205,1,1,0,1,0,0,1.0
194125,194126,0,0.482771,0.474048,0.138422,0.138462,0.121270,0.194426,1,1,1,0,0,1,1.0
66019,66020,0,3.532505,0.598616,0.138422,0.507692,0.130072,0.000039,1,1,0,1,0,0,1.0
45332,45333,0,0.156613,0.792388,0.103964,0.200000,0.087909,0.194426,1,1,0,1,0,0,1.0
108100,108101,0,1.703943,0.477509,0.103964,0.400000,0.187506,0.194426,1,1,0,1,0,0,1.0


In [28]:
# reset index
df8=df8.reset_index(drop=True)

# create ranking order
df8['ranking']=df8.index+1    #(para ñ começar do zero)

df8['precision_at_k']=df8['response'].cumsum()/df8['ranking']

df8[['id','response','score','ranking','precision_at_k']].head(10)

,id,response,score,ranking,precision_at_k
0,110250,0,1.000000,1,0.000000
1,194126,1,1.000000,2,0.500000
2,66020,0,1.000000,3,0.333333
3,45333,0,1.000000,4,0.250000
4,108101,0,1.000000,5,0.200000
5,334995,1,1.000000,6,0.333333
6,174463,1,1.000000,7,0.428571
7,43904,0,1.000000,8,0.375000
8,368460,1,0.857143,9,0.444444
9,37995,0,0.857143,10,0.400000


In [ ]:
# def precision_at_k(data,k=2000):   # k poderia ser outro valor como 20000
#     # reset index
#     df8=df8.reset_index(drop=True)

#     # create ranking order
#     df8['ranking']=df8.index+1    #(para ñ começar do zero)

#     df8['precision_at_k']=df8['response'].cumsum()/df8['ranking']
    
#     return data.loc[k,'precision_at_k']

In [38]:
# compute precision at k
precision_at_50=precision_at_k(df8,k=50)
precision_at_50

0.3137254901960784

In [39]:
# compute recall at k
recall_at_50=recall_at_k(df8,k=50)
recall_at_50

0.0017139796464916979

# **9 Deploy to production**

## **9.1 Health insurance class**

In [ ]:
class HealthInsurance:
    
    def __init__(self):        
        self.home_path=''
        self.annual_premium_scaler =            pickle.load(open(self.home_path + ''))
        self.age_scaler =                       pickle.load(open(self.home_path + ''))
        self.vintage_scaler =                   pickle.load(open(self.home_path + ''))
        self.target_encode_gender_scaler =      pickle.load(open(self.home_path + ''))
        self.target_encode_region_code_scaler = pickle.load(open(self.home_path + '')) 
        self.fe_policy_sales_channel_scaler =   pickle.load(open(self.home_path + ''))
        
        
    def data_cleaning(df1):        
#         rename columns
        cols_new=['id','gender','age','driving_license','region_code','previously_insured','vehicle_age',
                 'vehicle_damage','annual_premium','policy_sales_channel','vintage','response']
#         rename
        df1.columns=cols_new
    
        return df1
    
    
    def feature_engineering(self,df2):        
#         vehicle damage number
        df2['vehicle_damage']=df2['vehicle_damage'].apply(lambda x: 1 if x == 'Yes' else 0)
    
#         vehicle age
        df2['vehicle_age']=df2['vehicle_age'].apply(lambda x: 'over_2_years' if x == '> 2 Years' 
                                            else 'between_1_2_years' if x == '1-2 Year'
                                            else 'below_1_year')
    
    
    def data_preparation(df5):       
#         annual_premium - standardscaler
        df5['annual_premium']=self.annual_premium_scaler.transform(df5[['annual_premium']].values)
    
#         age - minmaxScaler
        df5['age']=self.age_scaler.transform(df5[['age']].values)
    
#         vintage - minmaxscaler
        df5['vintage']=self.vintage_scaler.fit_transform(df5[['vintage']].values)
        
#         gender - target encoding
        df5.loc[:,'gender']=df5['gender'].map(self.target_encode_gender_scaler)

#         region_code - target encoding
        df5.loc[:,'region_code']=df5['region_code'].map(self.target_encode_region_code_scaler)
    
#         vehicle_age - one hot encoding/frequency encoding
        df5=pd.get_dummies(df5,prefix='vehicle_age',columns=['vehicle_age'])
    
#         policy_sales_channel - frequency encoding
        df5.loc[:,'policy_sales_channel']=df5['policy_sales_channel'].map(self.fe_policy_sales_channel_scaler)
    
#         feature selection
        cols_selected=['annual_premium','vintage','age','region_code','vehicle_damage','previously_insured',
                      'policy_sales_channel']
    
        return df5[cols_selected]
    
    
    def get_prediction(self,model,original_data,test_data):   # test_data = produção
#         model prediction
        pred=model.predict_proba(test_data)
    
#         join prediction into original data
        original_data['score']=pred                  # score = prediction
    
        return original_data.to_json(orient='records',date_format='iso')

## **9.2 API handler**

In [ ]:
import pickle
import pandas as pd
from flask import Flask,request,Response
from healthinsurance import HealthInsurance


# loading model
path= '' ...health_insurance_cross-sell
model=pickle.load(open(path + '...model_health_insurance.pkl','rb'))


# rabitmq
# redis


# initial API
app=Flask(__name__)   # classe flask instanciada

@app.route('/predict',methods=['POST'])   # api apenas recebe requisições, predict é o endpoint
def health_insurance_predict():
    test_json=request.get_json()
    
    if test_json:  # theres date
        if isinstance(test_json,dict): # unique example         dicionário significa que foi recebido apenas uma linha 
            test_raw=pd.DataFrame(test_json,index=[0])  # ñ necessariamente precisa virar um dataframe (numpy,array etc)
                                                        # index=0 pands precisa da indicação da linha inicial pra criar o df
        else: # multiple example
            test_raw=pd.DataFrame(test_json,columns=test_json[0].keys())  # json significa q foi recebido várias linhas, nome das colunas são as chaves do json
            
#         instantiate health insurance class
        pipeline=HealthInsurance()
    
#         data cleaning
        df1=pipeline.data_cleaning(test_raw)
    
#         feature engineering
        df2=pipeline.feature_engineering(df1)
    
#         data preparation
        df3=pipeline.data_preparation(df2)
    
#         prediction
        df_response=pipeline.get_prediction(model,test_raw,df3)
    
        return df_response
    
    else:
        return Response( '{}', status=200, mimetype='application/json')
        
if __name__ == '__main__':
    app.run('0.0.0.0',debug=True)

## **9.3 API tester**

In [1]:
import requests

In [ ]:
# loading test dataset
df_test=x_validation
df_test['response']=y_validation  # o certo é ter o erro ao invés da response

In [ ]:
df_test=df_test.sample(10)

In [ ]:
# convert dataframe to json
data=json.dumps(df_test.to_dict(oriente='records'))

In [ ]:
data

In [ ]:
# API call
# url='http://0.0.0.0:5000/predict'                            # teste local
url='https://health-insurance-model.herokuapp.com/predict'     # teste produção
header={'Content-type':'application/json'}                     # indicando pra api q to passando um json

r=requests.post(url,data=data,headers=header)
print('Status Code {}'.format (r.status_code))

In [ ]:
d1=pd.DataFrame(r.json(),columns=r.json()[0].keys())
d1.sort_values('score',ascending=False).head()